# SAT Solver Assignment
Group 69<br>
**Knowledge Representation**<br>
November 2024

****

## Preperations

*Everything we need to have before we can build the SAT Solver.*

**Imports** - Use this cell for all imports.

In [2]:
# import modules
import math
from tqdm.autonotebook import tqdm
from collections import Counter


C:\Users\woute\AppData\Local\Temp\ipykernel_14348\964927310.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


**Data** - Import the needed data. The data needs to be in DIMACS format. Each clause in DIMACS CNF consists of:

- A series of positive or negative literals (numbers) that represent a Boolean variable or its negation.
- Each clause ends with 0.
- Each variable in DIMACS CNF is a positive integer, with unique integers representing each Boolean variable.

For Sudoku, we use a triplet `(r, c, n)`:
- `r` for the row (1 to 9)
- `c` for the column (1 to 9)
- `n` for the number (1 to 9)

In [3]:
# Geeft een file met 91 sudoku's terug, sudokus[0] is de eerste sudoku met len(sudokus[0]) == 81 (9x9=81 karakters)
with open('./top91.sdk.txt') as f:
    sudokus = f.read().splitlines()

FileNotFoundError: [Errno 2] No such file or directory: './top91.sdk.txt'

*You are free to choose any programming language you fancy, but we must be able to run your SAT solver with the command SAT -Sn inputfile , for example: SAT -S2 sudoku_nr_10 , where SAT is the (compulsory) name of your program, n=1 for the basic DP and n=2 or 3 for your two other strategies, and the input file is the concatenation of all required input clauses (in your case: sudoku rules + given puzzle).*

****

## Part 1 - Building the SAT Solver

**The Rules of Sudoku**
1. Each number (1-9) must appear exactly once in each row.
2. Each number must appear exactly once in each column.
3. Each number must appear exactly once in each 3x3 subgrid.

**The Data of Sudoku in DIMACS**

In [4]:
# see util.py
from util import *

**Functions to Assign Obvious Values** - Fill in the single and pure literal clauses.

In [ ]:
# see literalmethods.py

**Heuristics** - Two different heuristics of our choice

In [ ]:
# see hueristics.py

**DPLL Algorithm** - DPLL recursive algorithm function,  without heuristics, to perform the fucntions above and recursively gives values to literals.

In [ ]:
# see SAT.py

*give (2)+(3) as input to (1) and return the solution to the given puzzle. This output should again be a DIMACS file, but containing only the truth assignment to all variables (729 for Sudoku, different for other SAT problems). If your input file is called 'filename', then make sure your outputfile is called 'filename.out'. If there is no solution (inconsistent problem), the output can be an empty file. If there are multiple solutions (eg. non-proper Sudoku) you only need to return a single solution.*

**SAT Solver** - Solves the Sudoku

Extra information about the code:
Each cell in a Sudoku puzzle can be represented by a unique variable based on its row, column, and possible number. The encoding formula for each variable is:

`variable=(i+1)×100+(j+1)×10+value`<br>

where:<br>

`i is the row index (0 to 8)`<br>
`j is the column index (0 to 8)`<br>
`value is the number in the cell (1 to 9)`<br>

This formula converts each cell and its possible values into unique numbers by combining the row, column, and value in a single integer. For example, if a cell at row 0, column 0 can be a 5, the encoded variable would be:<br>

`variable=(0+1)×100+(0+1)×10+5=115`<br>

In [7]:
from SAT import solve_sudoku

**Test Sudoku** - Try and see if it works

In [ ]:
# Load the Sudoku rules and the puzzle
filename = './sudoku/sudoku-rules-16x16.txt'
rules = load_rules(filename)
# print(rules)

# Load the puzzles from the file and select the second puzzle (index 1)
puzzles, size = load_sudoku('./sudoku/16x16.txt')
puzzle = puzzles[0]
print(puzzle)

# Solve the Sudoku puzzle
solution = solve_sudoku(rules, puzzle, size, heuristic='chb')

print('puzzle:     ', puzzle)
print('solution:   ', solution)

# If a solution is found, print the solved Sudoku grid
if solution:
    for row in solution:
        print(row)
        if 0 in row:
            print("PANIEK")
else:
    print("No solution found")
    # 263 385 517 566 647
    
# solution:    [[1, 16, 13, 11, 12, 2, 3, 4, 9, 10, 5, 8, 6, 14, 15, 7], [4, 14, 6, 10, 5, 7, 15, 8, 1, 2, 3, 12, 11, 13, 9, 16], [5, 2, 12, 7, 6, 9, 16, 11, 15, 13, 14, 4, 10, 1, 8, 3], [3, 9, 15, 8, 1, 10, 14, 13, 7, 6, 16, 11, 2, 4, 12, 5], [9, 4, 2, 6, 13, 3, 1, 15, 10, 16, 11, 14, 5, 8, 7, 12], [8, 12, 7, 14, 11, 6, 9, 5, 4, 15, 1, 2, 16, 3, 10, 13], [16, 15, 1, 13, 14, 4, 7, 10, 5, 12, 8, 3, 9, 11, 2, 6], [11, 3, 10, 5, 16, 8, 12, 2, 6, 9, 13, 7, 14, 15, 1, 4], [2, 6, 3, 16, 4, 1, 8, 14, 11, 5, 7, 15, 12, 10, 13, 9], [7, 11, 14, 1, 3, 12, 10, 16, 2, 4, 9, 13, 8, 6, 5, 15], [15, 8, 5, 9, 7, 13, 11, 6, 12, 3, 10, 16, 4, 2, 14, 1], [10, 13, 4, 12, 15, 5, 2, 9, 8, 14, 6, 1, 7, 16, 3, 11], [14, 1, 9, 4, 10, 16, 6, 3, 13, 8, 12, 5, 15, 7, 11, 2], [13, 7, 8, 15, 9, 14, 5, 1, 16, 11, 2, 6, 3, 12, 4, 10], [12, 10, 16, 3, 2, 11, 4, 7, 14, 1, 15, 9, 13, 5, 6, 8], [6, 5, 11, 2, 8, 15, 13, 12, 3, 7, 4, 10, 1, 9, 16, 14]]


puzzledim: 16
[[1, 0, 13, 0, 0, 0, 0, 4, 0, 10, 5, 8, 0, 0, 0, 0], [0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 16], [0, 2, 0, 7, 6, 0, 16, 11, 15, 0, 0, 4, 0, 0, 0, 0], [3, 9, 15, 0, 1, 10, 0, 13, 7, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 6, 0, 3, 1, 0, 0, 0, 11, 0, 5, 8, 0, 12], [8, 12, 7, 14, 0, 6, 9, 0, 0, 15, 0, 0, 0, 0, 0, 13], [0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 10, 0, 16, 8, 12, 0, 0, 0, 0, 7, 14, 0, 1, 4], [2, 6, 0, 16, 4, 0, 0, 0, 0, 5, 7, 15, 0, 10, 0, 0], [0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0], [15, 0, 0, 0, 0, 0, 11, 0, 0, 3, 10, 0, 4, 2, 14, 1], [10, 0, 4, 12, 0, 5, 0, 0, 0, 14, 6, 0, 7, 0, 3, 0], [0, 0, 0, 0, 0, 0, 0, 3, 13, 0, 12, 5, 0, 7, 11, 2], [0, 0, 0, 0, 9, 0, 0, 1, 16, 11, 0, 6, 3, 0, 4, 0], [12, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0], [0, 0, 0, 0, 8, 15, 13, 0, 3, 0, 0, 0, 0, 9, 0, 14]]
